In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.GNN_models import Model_GCN, Model_GAT, APPNP, DAGNN
from utils.preprocess import norm_adj, norm_hop_adj, data_split, clean_A, remove_edge_pts
from torch.utils.data import DataLoader
import numpy as np
from collections import Counter
import random
import dgl.data
import dgl
import time
from sklearn import metrics
from matplotlib import pyplot as plt

Using backend: pytorch


In [2]:
def evaluate_f1(model, x, y, val_mask, test_mask):
    model.eval()
    with torch.no_grad():
        y_pred = model(x)
        y_pred_val = y_pred[val_mask]
        y_pred_test = y_pred[test_mask]
        pred_val = y_pred_val.argmax(1)
        pred_test = y_pred_test.argmax(1)
        val_f1 = metrics.f1_score(y[val_mask].cpu().numpy(), pred_val.cpu().numpy(), average='macro')
        test_f1 = metrics.f1_score(y[test_mask].cpu().numpy(), pred_test.cpu().numpy(), average='macro')
        return val_f1, test_f1
    
def evaluate_acc(model, x, y, val_mask, test_mask):
    model.eval()
    with torch.no_grad():
        y_pred = model(x)
        y_pred_val = y_pred[val_mask]
        y_pred_test = y_pred[test_mask]
        pred_val = y_pred_val.argmax(1)
        pred_test = y_pred_test.argmax(1)
        val_acc = (y[val_mask] == pred_val).float().mean().item()
        test_acc = (y[test_mask] == pred_test).float().mean().item()
        return val_acc, test_acc

In [3]:
def train(model, x, y, train_mask, val_mask, test_mask, args):
    device = args.device
    model.to(device)
    x = x.to(device)
    y = y.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    criterion = nn.CrossEntropyLoss().to(device)
    best_val_acc = 0
    best_test_acc = 0
    count = 0
    
    tic = time.time()
    for epoch in range(args.num_iter):    
        model.train()
        y_pred = model(x)
        loss = criterion(y_pred[train_mask], y[train_mask])
        val_acc, test_acc = evaluate_acc(model, x, y, val_mask, test_mask)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            count = 0
        elif count >= args.patience:
            break
        else:
            count += 1
    print('time duration = {:.3f}, best_val_acc = {:.3f}, best_test_acc = {:.3f}'.
          format(time.time() - tic, best_val_acc, best_test_acc))
    #model = model.to('cpu')
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return best_test_acc, model 

In [4]:
def test_acc(x, y, A, train_mask, val_mask, test_mask, args):
    accs = []
    for _ in range(args.num_test):
        if args.model == 'GCN':
            n_dims = [args.n_in] + [args.n_hid] * args.hop + [args.n_out]
            #print('n_dims: ', n_dims)
            model = Model_GCN(n_dims, A, args.dropout)
        elif args.model == 'GAT':
            n_dims = [args.n_in] + [args.n_hid] * args.hop + [args.n_out]
            #print('n_dims: ', n_dims)
            n_heads = [args.num_heads] * args.hop + [args.num_out_heads]
            #print('n_heads: ', n_heads)
            model = Model_GAT(n_dims, n_heads, A, args.dropout, args.dropout2)
        elif args.model == 'APPNP':
            model = APPNP(args.n_in, args.n_hid, args.n_out, A, args.dropout, args.dropout2, args.hop, args.alpha)
        elif args.model == 'DAGNN':
            model = DAGNN(args.n_in, args.n_hid, args.n_out, A, args.hop, args.dropout)
        acc, _ = train(model, x, y, train_mask, val_mask, test_mask, args)
        accs.append(acc) 
    accs = np.array(accs)
    print('test acc = ', accs.mean())
    return accs

In [5]:
def main(args):
    if args.data == 'Citeseer':
        dataset = dgl.data.CiteseerGraphDataset()
    elif args.data == 'Pubmed':
        dataset = dgl.data.PubmedGraphDataset()
    elif args.data == 'Coauthor-CS':
        dataset = dgl.data.CoauthorCSDataset()
    else:
        dataset = dgl.data.CoraGraphDataset()
    g = dataset[0]
    A = g.adjacency_matrix()
    A = clean_A(A)
    x = g.ndata['feat']
    y = g.ndata['label']

    args.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    args.n_in, args.n_out = x.size(1), len(set(y.tolist()))
    if args.data == 'Coauthor-CS' or args.random_label_split:
        if args.data_load:
            train_mask = torch.load('./label_split/train_mask_' + args.data + str(args.test_id) + '.pt')
            val_mask = torch.load('./label_split/val_mask_' + args.data + str(args.test_id) + '.pt')
            test_mask = torch.load('./label_split/test_mask_' + args.data + str(args.test_id) + '.pt')
        else:
            # generate label split randomly
            train_mask, val_mask, test_mask = data_split(x, y, training_samples=args.num_train, val_samples=args.num_val)
            torch.save(train_mask, './label_split/train_mask_' + args.data + str(args.test_id) + '.pt')
            torch.save(val_mask, './label_split/val_mask_' + args.data + str(args.test_id) + '.pt')
            torch.save(test_mask, './label_split/test_mask_' + args.data + str(args.test_id) + '.pt')
    else:
        train_mask = g.ndata['train_mask']
        val_mask = g.ndata['val_mask']
        test_mask = g.ndata['test_mask']
    accs = test_acc(x, y, A, train_mask, val_mask, test_mask, args)
    print('test accuracy (mean, std): ', accs.mean(), accs.std())
    acc = remove_edge_pts(accs, pct=args.filter_pct)
    print('test accuracy (mean, std) after filter: ', acc.mean(), acc.std())
    return accs

In [6]:
# 0.5, 0.5, 0.005, 5e-4 for AmazonCoBuy 0.8, 0.5 ,0.01, 5e-4 for Pubmed, 0.8, 0.6, 0.01, 5e-4 for citeseer
class config:
    data = 'Cora'
    model = 'APPNP'
    n_in = 0
    n_hid = 8
    n_out = 0
    num_heads = 8
    num_out_heads = 1
    device = 'cpu'
    dropout = 0.6
    dropout2 = 0.6
    #GCN_dropout = 0.5
    #GAT_dropout = 0.6
    #GAT_attn_dropout = 0.6
    #APPNP_dropout1 = 0.5
    #APPNP_dropout2 = 0.5
    #DAGNN_dropout = 0.8
    #GTreeConv_dropout1 = 0.6
    #GTreeConv_dropout2 = 0.6
    learning_rate = 0.005
    weight_decay = 1e-3
    patience = 100
    num_iter = 1000
    num_test = 30
    hop = 1
    alpha = 0.1 # used by APPNP only
    random_label_split = False
    num_train = 20 # for random label split only
    num_val = 30 # for random label split only
    data_load = False # load the saved label split to rerun the test (for reproduce purpose)
    test_id = 1 # number of the test, only used to record the ith number of the random label split (for reproduce purpose)
    filter_pct = 0.1 # remove the top and bottom filer_pct points before obtaining statistics of test accuracy

In [7]:
args = config()
accs = main(args)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
self_loop # =  0
time duration = 14.869, best_val_acc = 0.762, best_test_acc = 0.768
time duration = 26.690, best_val_acc = 0.772, best_test_acc = 0.779
time duration = 22.758, best_val_acc = 0.800, best_test_acc = 0.789
time duration = 17.741, best_val_acc = 0.780, best_test_acc = 0.795
time duration = 17.243, best_val_acc = 0.736, best_test_acc = 0.750
time duration = 28.284, best_val_acc = 0.788, best_test_acc = 0.768
time duration = 23.038, best_val_acc = 0.774, best_test_acc = 0.786
time duration = 22.121, best_val_acc = 0.772, best_test_acc = 0.788
time duration = 14.324, best_val_acc = 0.784, best_test_acc = 0.798
time duration = 13.821, best_val_acc = 0.774, best_test_acc = 0.790
time duration = 24.179, best_val_acc = 0.786, best_test_acc = 0.796
time duration = 25.181, best_val_acc = 0.792, best_te